数据预处理, 将WiderPerson.zip中的数据转为yolo可以训练的格式

In [1]:
import os
import shutil

import cv2
import tqdm
import matplotlib.pyplot as plt

In [2]:
wider_person_dir = os.path.join(os.path.abspath("."), "WiderPerson")
annotations_dir = os.path.join(wider_person_dir, "Annotations")
images_dir = os.path.join(wider_person_dir, "Images")

# 转换之后的路径
person_data_dir = os.path.join(os.path.abspath("."), "person_data")

In [3]:
def convert_2_yolo(image_id, save_path_prefix="./person_data/labels/train"):
    """读取标注文件, 转为yolo格式"""
    origin_label_file = os.path.join(annotations_dir, f"{image_id}.jpg.txt")
    with open(origin_label_file, "r") as f:
        lines = f.readlines()

    lines = [line.strip() for line in lines]

    # 获取标注
    boxes = lines[1:]
    boxes = [box.split() for box in boxes]

    # 读取标注文件对应的图片
    img_file = os.path.join(images_dir, f"{image_id}.jpg")
    img = cv2.imread(img_file)
    # 转为yolo格式: 类别 id, x_center, y_center, width, height, 归一化到0-1, 保留6位小数
    yolo_boxes = []
    img_h, img_w, _ = img.shape
    for box in boxes:
        class_label = int(box[0]) - 1
        x1, y1, x2, y2 = [int(i) for i in box[1:]]
        x_center = round((x1 + x2) / 2 / img_w, 6)
        y_center = round((y1 + y2) / 2 / img_h, 6)
        width = round((x2 - x1) / img_w, 6)
        height = round((y2 - y1) / img_h, 6)
        yolo_boxes.append([class_label, x_center, y_center, width, height])

    if not os.path.exists(save_path_prefix):
        os.makedirs(save_path_prefix)

    # 写入txt文件
    yolo_label_file = os.path.join(save_path_prefix, f"{image_id}.txt")
    with open(yolo_label_file, "w") as f:
        for yolo_box in yolo_boxes:
            f.write(" ".join([str(i) for i in yolo_box]) + "\n")

    if os.path.exists(yolo_label_file):
        return yolo_label_file

In [4]:
def yolo_draw(img, yolo_label_file):
    """根据yolo格式的标注文件, 在图片上绘制"""
    img_copy = img.copy()
    color_dict = {
        0: (255, 0, 0),
        1: (0, 255, 0),
        2: (0, 0, 255),
        3: (255, 255, 0),
        4: (0, 255, 255),
    }
    with open(yolo_label_file, "r") as f:
        lines = f.readlines()

        lines = [line.strip() for line in lines]

        boxes = [line.split() for line in lines]

        for box in boxes:
            class_label = int(box[0])
            x_center, y_center, width, height = [float(item) for item in box[1:]]
            x1 = int((x_center - width / 2) * img_copy.shape[1])
            y1 = int((y_center - height / 2) * img_copy.shape[0])
            x2 = int((x_center + width / 2) * img_copy.shape[1])
            y2 = int((y_center + height / 2) * img_copy.shape[0])
            cv2.rectangle(img_copy, (x1, y1), (x2, y2), color_dict[class_label], 2)
            cv2.putText(img_copy, str(class_label), (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, color_dict[class_label], 2)
    
    plt.imshow(img_copy[:,:,::-1])
    plt.axis("off")
    plt.show()

In [5]:
# test_img_id = "000045"

In [6]:
# convert_2_yolo(test_img_id)

In [7]:
# img_file = os.path.join(images_dir, f"{test_img_id}.jpg")
# img = cv2.imread(img_file)
# plt.imshow(img[:,:,::-1])
# plt.axis("off")

In [8]:
# yolo_draw(img, convert_2_yolo(test_img_id))

In [5]:
images_train_dir = os.path.join(person_data_dir, "images", "train")
images_val_dir = os.path.join(person_data_dir, "images", "val")
if not os.path.exists(images_train_dir):
    os.makedirs(images_train_dir)
if not os.path.exists(images_val_dir):
    os.makedirs(images_val_dir)

labels_train_dir = os.path.join(person_data_dir, "labels", "train")
labels_val_dir = os.path.join(person_data_dir, "labels", "val")
if not os.path.exists(labels_train_dir):
    os.makedirs(labels_train_dir)
if not os.path.exists(labels_val_dir):
    os.makedirs(labels_val_dir)

In [6]:
! tree ./person_data

./person_data
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories, 0 files


In [7]:
# 获取所有训练图片的文件名
with open(os.path.join(wider_person_dir, "train.txt"), "r") as f:
    train_img_file_names = f.readlines()
    train_img_file_names = [x.strip() for x in train_img_file_names]

# 获取所有验证图片的文件名
with open(os.path.join(wider_person_dir, "val.txt"), "r") as f:
    val_img_file_names = f.readlines()
    val_img_file_names = [x.strip() for x in val_img_file_names]

In [8]:
print("train_img_file_names: ", len(train_img_file_names))
print("val_img_file_names: ", len(val_img_file_names))

train_img_file_names:  8000
val_img_file_names:  1000


In [9]:
for img_file_name in tqdm.tqdm(train_img_file_names, desc="train"):
    yolo_label_file = convert_2_yolo(img_file_name, labels_train_dir)
    if yolo_label_file:
        img_file = os.path.join(images_dir, f"{img_file_name}.jpg")
        shutil.copy(img_file, os.path.join(images_train_dir, f"{img_file_name}.jpg"))

for img_file_name in tqdm.tqdm(val_img_file_names, desc="val"):
    yolo_label_file = convert_2_yolo(img_file_name, labels_val_dir)
    if yolo_label_file:
        img_file = os.path.join(images_dir, f"{img_file_name}.jpg")
        shutil.copy(img_file, os.path.join(images_val_dir, f"{img_file_name}.jpg"))

val: 100%|██████████| 1000/1000 [00:02<00:00, 458.03it/s]


In [10]:
# 检查文件数量
print('train_img_file_names:', len(os.listdir('./person_data/images/train')))
print('val_img_file_names:', len(os.listdir('./person_data/images/val')))
print('train_label_file_names:', len(os.listdir('./person_data/labels/train')))
print('val_label_file_names:', len(os.listdir('./person_data/labels/val')))

train_img_file_names: 8000
val_img_file_names: 1000
train_label_file_names: 8000
val_label_file_names: 1000
